# Pruebas con las bases de datos



## Preamble 

In [ ]:
import $file.sparksession
import sparksession._
import spark.implicits._
import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._, func._

In [ ]:
//import $ivy.`org.vegas-viz::vegas:0.3.9`

In [ ]:
//val sqlContext = SparkSession.builder().getOrCreate();

## DataFrame

In [ ]:
def readCsv(input: String ): DataFrame ={
    spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("data/" + input + ".csv")
}

In [ ]:
val temperatureByCountry : DataFrame = readCsv("GlobalLandTemperaturesByCountry")
val disasterEconomic: DataFrame = readCsv("economic-damage-from-natural-disasters").drop($"Code")
val deathByCountry= readCsv("deaths-natural-disasters-ihme").withColumn("Deaths", $"Deaths".cast("Long"))

In [ ]:
val disasterNumber = readCsv("number-of-natural-disaster-events").drop($"Code")

In [ ]:
val disasterDeath : DataFrame = readCsv("number-of-deaths-from-natural-disasters").drop($"Code")
val deathPercentByCountry: DataFrame = readCsv("share-deaths-from-natural-disasters")
                                .withColumn("Deaths(Percent) (%)",format_number($"Deaths(Percent) (%)", 2).cast("Double"))


In [ ]:
val earthquakes : DataFrame = readCsv("significant-earthquakes").drop($"Code")

In [ ]:
val volcano : DataFrame = readCsv("significant-volcanic-eruptions").drop($"Code")

## Para seccion 3.2 Spark

In [ ]:
//Para seccion 3.2.1
//dataset de prueba
val lista = List((1,3,"hola"),(2,4,"tarde"),
        (3,2,"noche"),(4,5,"buenas"),(5,10,"mundo"))
val listDS = lista.toDS

In [ ]:
//quedar con los elementos que tenga el 
// primer componente mayor que 2
lista.filter(_._1 > 2)
//los que tenga el elemento en la primera columna 
//mayor que 2
listDS.filter($"_1" > 2)
// quedar con el segundo y tercer componente
lista.map{ case(n,f,s) => (f,s)}
//Quedar con la segunda y tercera columna
listDS.select($"_2",$"_3").collect

In [ ]:
//para seccion 3.2.2
disasterNumber.filter($"Entity" ==="All natural disasters")
val temperatureByCountryYear = temperatureByCountry
                                .withColumn("dt", date_format(col("dt"), "yyyy")
                                .cast("Int"))
val temperatureAVGTotal = temperatureByCountryYear
                    .groupBy($"dt")
                    .avg("AverageTemperature")
                    .orderBy($"dt")
temperatureAVGTotal.map(_.getAs[Double](1))

In [ ]:
val transformacion = disasterNumber.select($"Entity", $"Year", $"Number")

In [ ]:
transformacion.show

In [ ]:
val disasterNumberDeath = disasterNumber
        .join(disasterDeath, Seq("Entity","Year"))

In [ ]:
disasterNumber
    .filter($"Entity" ==="All natural disasters")
    .orderBy($"Year")
    .select($"Year",$"Number")
    .map(_.getInt(0))

In [ ]:
disasterNumber
    .filter($"Entity" ==="All natural disasters")
    .show

In [ ]:
disasterNumber
    .groupBy($"Year")
    .count
    .show

In [ ]:
//Para seccion 3.2.3
transformacion.explain

In [ ]:
transformacion.explain(extended = true)

In [ ]:
//para seccion 3.2.4
transformacion.rdd

## algunas queries

In [ ]:
disasterNumber.filter($"Entity" =!="All natural disasters")
         .groupBy($"Year")
         .sum("Number")
         .orderBy($"Year")
         .show

disasterNumber.filter($"Entity" ==="All natural disasters")
         .show


disasterNumber.filter($"Entity" =!="All natural disasters")
         .groupBy($"Year", $"Entity")
         .sum("Number")
         .orderBy($"Year".desc)
         .show

In [ ]:
temperatureByCountry.filter($"AverageTemperature" > -1000000)
            .groupBy($"dt")
            .avg("AverageTemperature")
            .orderBy($"dt")
            .limit(20)
            .show

## Queries con explicaciones

In [ ]:
//Relacionar numero de desastres con sus daños economicos

val disasterNumberDamage =disasterNumber.join(disasterEconomic, Seq("Entity","Year"))
disasterNumberDamage.filter($"Entity" =!="All natural disasters").limit(10).show

In [ ]:
//Creo que no funciona porque en ambos casos, en el campo Code es null

disasterNumber.join(disasterEconomic, Seq("Entity","Code","Year")).limit(10).show

In [ ]:
// Modificar la columna dt de tipo timestamp (yyyy-MM-dd hh-mm-ss) a solo de años 

val temperatureByCountryYear:DataFrame = temperatureByCountry.withColumn("dt", date_format(col("dt"), "yyyy"))

In [ ]:
// tabla con temperetura media del año por pais 
//ordenado primero por el año y luego por pais
val temperatureAVG = temperatureByCountryYear
                    .groupBy($"dt",$"Country")
                    .avg("AverageTemperature")
                    .orderBy($"dt", $"Country")
temperatureAVG.limit(10).show

In [ ]:
//Relacionar el numero total de muertos con numero de desastres

val disasterNumberDeath =disasterNumber.join(disasterDeath, Seq("Entity","Year"))
disasterNumberDeath.limit(10).show


In [ ]:
//Relacionar numero de muertos por pais con su porcentage

val deathNumberPercent =deathByCountry.join(deathPercentByCountry, Seq("Entity","Year"))
deathNumberPercent.filter($"Year">2000).filter($"Entity" === "Haiti").limit(10).show

## Solución con Plotly

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.2`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._


In [ ]:

//purebas

// val consulta= disasterNumber.filter($"Entity" =!="All natural disasters")
//          .groupBy($"Year",$"Entity")
//          .sum("Number")
//          .orderBy($"Year")
// val x=consulta.select($"Year").collectAsList.toArray.mkString("!!!").replace("[","").replace("]","").split("!!!").toSeq
// val y = consulta.select($"sum(Number)").collectAsList.toArray.mkString("!!!").replace("[","").replace("]","").split("!!!").toSeq
// Bar(x,y).plot()

// val trace1 = Scatter(x,y)


// trace1.plot()

In [ ]:
val disasterNumber = readCsv("number-of-natural-disaster-events").drop($"Code")

In [ ]:
// 1.¿Cómo evoluciona el número de desastres naturales según avanza los años?
val aux = disasterNumber.filter($"Entity" ==="All natural disasters").orderBy($"Year").select($"Year",$"Number")
Bar(aux.map(_.getInt(0)).collect.toSeq,
    aux.map(_.getInt(1)).collect.toSeq).plot()

In [ ]:
//Para seccion 3.3
val aux1 = aux.collect
Bar(aux1.map(_.getInt(0)).toSeq,
    aux1.map(_.getInt(1)).toSeq).plot()

In [ ]:
// 2.¿Cómo evoluciona el daño económico causado por los desastres naturales?
val aux = disasterEconomic.filter($"Entity" ==="All natural disasters").orderBy($"Year").select($"Year",$"Money")
Bar(aux.map(_.getInt(0)).collect.toSeq,aux.map(_.getLong(1)).collect.toSeq).plot()

In [ ]:
//3.¿Existe alguna relación entre el número de desastres naturales con el daño económico?

val diasasterNumberEconomic=disasterNumber.join(disasterEconomic, Seq("Entity","Year"))
val trace1= Scatter(
        diasasterNumberEconomic.map(_.getAs[Int]("Number")).collect.toSeq, 
        diasasterNumberEconomic.map(_.getAs[Long]("Money")).collect.toSeq,
        mode = ScatterMode(ScatterMode.Markers)
        )
        .plot()

In [ ]:
// Modificar la columna dt de tipo timestamp (yyyy-MM-dd hh-mm-ss) a solo de años 

val temperatureByCountryYear:DataFrame = temperatureByCountry.withColumn("dt", date_format(col("dt"), "yyyy").cast("Int"))

In [ ]:
// 4.¿Cómo evoluciona la temperatura media anual global?
val temperatureAVGTotal = temperatureByCountryYear
                    .groupBy($"dt")
                    .avg("AverageTemperature")
                    .orderBy($"dt")
Scatter(
    temperatureAVGTotal.map(_.getInt(0)).collect.toSeq,
    temperatureAVGTotal.map(_.getAs[Double](1)).collect.toSeq
).plot()

In [ ]:
// 5.¿Cómo evoluciona la temperatura media anual de cada país?
val temperatureAVG = temperatureByCountryYear
                    .groupBy($"dt",$"Country")
                    .avg("AverageTemperature")
                    .orderBy($"dt", $"Country")
                    


In [ ]:
//Cojo dos paises como ejemplo
val albania =temperatureAVG.filter($"Country"==="Albania")
val trace1 = Bar(
  albania.map(_.getAs[Int]("dt")).collect.toSeq,
  albania.map(_.getAs[Double]("avg(AverageTemperature)")).collect.toSeq,
  name = "Albania"
)

val andorra =temperatureAVG.filter($"Country"==="Andorra")
val trace2 = Bar(
  andorra.map(_.getAs[Int]("dt")).collect.toSeq,
  andorra.map(_.getAs[Double]("avg(AverageTemperature)")).collect.toSeq,
  name = "Andorra"
)


val data = Seq(trace1, trace2)

data.plot()

// val layout = Layout( 
//   barmode = BarMode.Group
// )

// Plotly.plot("div-id", data, layout)

In [ ]:
//6.¿Existe alguna relación entre la temperatura media anual global 
//con el número de desastres naturales?
val aux1 = disasterNumber.filter($"Entity" ==="All natural disasters")

val aux = aux1.join(temperatureAVGTotal, aux1("Year")===temperatureAVG("dt"))

Scatter(
    aux.map(_.getAs[Double](4)).collect.toSeq,
    aux.map(_.getAs[Int](2)).collect.toSeq,
    mode = ScatterMode(ScatterMode.Markers)
).plot()

In [ ]:
//7.¿Cómo evoluciona el número de muertes anual 
//causados por los desastres naturales?
val aux = deathByCountry
       .groupBy($"Year")
       .sum("Deaths")
       .orderBy($"Year")
Bar(
    aux.map(_.getAs[String](0)).collect.toSeq,
    aux.map(_.getAs[Long](1)).collect.toSeq
).plot()

In [ ]:
//8.¿Cómo evoluciona la media del porcentaje de muertes?
val aux = deathPercentByCountry
       .groupBy($"Year")
       .avg("Deaths(Percent) (%)")
       .orderBy($"Year")
Bar(
    aux.map(_.getAs[String](0)).collect.toSeq,
    aux.map(_.getAs[Double](1)).collect.toSeq
).plot()

In [ ]:
//9.¿Cuál es el país con más muertes por los desastres naturales?

  deathByCountry
  .groupBy($"Entity")
  .sum("Deaths")
  .orderBy($"sum(Deaths)".desc)
  .show

In [ ]:
//10.¿Existe alguna relación entre el número de desastres naturales con el número 
//total de muertos anuales causados por los desastres naturales?

val disasterNumberDeath =disasterNumber.join(disasterDeath, Seq("Entity","Year")).filter($"Entity"==="All natural disasters")


Scatter(
        disasterNumberDeath.map(_.getAs[Int]("Number")).collect.toSeq, 
        disasterNumberDeath.map(_.getAs[Int]("Deaths")).collect.toSeq,
        mode = ScatterMode(ScatterMode.Markers)
        )
        .plot()

In [ ]:
// 11.¿Cómo evolucionan cada tipo de desastres naturales según avanza el tiempo?

 val aux = disasterNumber.filter($"Entity" =!="All natural disasters")
         .groupBy($"Year", $"Entity")
         .sum("Number")
         .orderBy($"Year")

//        val data= aux.groupByKey(_.getAs[String]("Entity"))
//          .mapGroups((key,value) => Scatter(value.map(_.getAs[Int]("Year")).toSeq,
//                                           value.map(_.getAs[Int]("Number")).toSeq,
//                                           name = key))

val trace1 = Scatter(
  aux.filter($"Entity" === "Flood").map(_.getInt(0)).collect.toSeq,
  aux.filter($"Entity" === "Flood").map(_.getLong(2)).collect.toSeq,
  name="Flood"
)


val trace2 = Scatter(
  aux.filter($"Entity" === "Extreme weather").map(_.getInt(0)).collect.toSeq,
  aux.filter($"Entity" === "Extreme weather").map(_.getLong(2)).collect.toSeq,
  name="Extreme weather"
)

val data = Seq(trace1, trace2)

data.plot()



In [ ]:
//12.¿Qué tipo de desastres naturales provoca mayores muertos?
disasterNumber.join(disasterDeath, Seq("Entity","Year"))
       .groupBy($"Entity")
       .sum("Deaths")
       .filter($"Entity" =!= "All natural disasters")
       .orderBy($"sum(Deaths)".desc).show

In [ ]:
//13.¿Qué tipo de desastres naturales provoca mayores daños económicos?
disasterEconomic
       .groupBy($"Entity")
       .sum("Money")
       .filter($"Entity" =!= "All natural disasters")
       .orderBy($"sum(Money)".desc)
       .show

In [ ]:
//14.¿Cuándo hubo más concentración de terremotos significativos?
Bar(
    earthquakes.filter($"Year">1700).map(_.getAs[Int](1)).collect.toSeq,
    earthquakes.filter($"Year">1700).map(_.getAs[Int](2)).collect.toSeq
).plot()

In [ ]:
//15.¿Cuándo hubo más concentración de erupciones volcánicas?
Bar(
    volcano.filter($"Year">1700).map(_.getAs[Int](1)).collect.toSeq,
    volcano.filter($"Year">1700).map(_.getAs[Int](2)).collect.toSeq
).plot()

## Para capitulo 4.2.1 Parquet

In [ ]:
val temperatureByCity = readCsv("GlobalLandTemperaturesByCity")

In [ ]:
//Guardar en formato parquet 
temperatureByCity.write.mode("overwrite").parquet("data/temperatureByCity.parquet")

In [ ]:
val csv = readCsv("GlobalLandTemperaturesByCity").select($"City")

In [ ]:
csv.collect

In [ ]:
val parquet: DataFrame = 
    spark.read.parquet("data/temperatureByCity.parquet").select($"City")

In [ ]:
parquet.collect

In [ ]:
csv.explain

In [ ]:
parquet.explain

In [ ]:
//Dataset, Dataframe, parquet
val people: DataFrame = spark.read.json("data/people.json")

In [ ]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class Person(name: String, age: Long)

val peopleDs: Dataset[Person] = people.as[Person]

In [ ]:
val ds = peopleDs.filter(_.age > 15)
ds.explain

In [ ]:
val csv = readCsv("GlobalLandTemperaturesByCity")

In [ ]:
val df= csv.select($"City", $"Country")

In [ ]:
val parq =  spark.read.parquet("data/temperatureByCity.parquet").select($"City",$"Country")

In [ ]:
val df_f = df.filter($"Country" === "Denmark")

In [ ]:
df_f.explain

In [ ]:
df_f.collect

In [ ]:
val parq_f = parq.filter($"Country" === "Denmark")

In [ ]:
parq_f.explain

In [ ]:
parq_f.collect

In [ ]:
//guardamos en memoria con partition
temperatureByCity
    .write.mode("overwrite")
    .partitionBy("Country")
    .parquet("data/country.parquet")

In [ ]:
//guardamos en memoria con repartition previo
temperatureByCity
    .repartition($"Country")
    .write.mode("overwrite")
    .partitionBy("Country")
    .parquet("data/countryre.parquet")

In [ ]:
//filter sin partition
val sin: DataFrame = 
    spark.read.parquet("data/temperatureByCity.parquet")
            .filter($"Country" === "Denmark" && $"AverageTemperature" >10)

In [ ]:
sin.show

In [ ]:
sin.explain

In [ ]:
//filter con partition
val con = spark.read.parquet("data/country.parquet")
            .filter($"Country" === "Denmark" && $"AverageTemperature" >10)

In [ ]:
con.show

In [ ]:
con.explain

## Pruebas con Evilplot
no funciona

In [ ]:
interp.repositories() ++= Seq(
  coursierapi.MavenRepository.of("https://dl.bintray.com/cibotech/public")
)


In [ ]:
import $ivy.`com.cibo::evilplot-repl:0.7.0`
def showPlot(plot: com.cibo.evilplot.geometry.Drawable) =
  Image.fromRenderedImage(plot.asBufferedImage, Image.PNG)

In [ ]:
// 1.	¿Cómo evoluciona el número de desastres naturales según avanza los años?
val aux = disasterNumber.filter($"Entity" ==="All natural disasters").orderBy($"Year").select($"Year",$"Number")


In [ ]:
import com.cibo.evilplot.colors.RGB
import com.cibo.evilplot.geometry.{Align, Drawable, Extent, Rect, Text}
import com.cibo.evilplot.plot._
import com.cibo.evilplot.plot.aesthetics.DefaultTheme.{DefaultFonts, DefaultTheme}
import com.cibo.evilplot.plot.renderers.BarRenderer



BarChart
  .custom(aux.map(_.getInt(1)).collect.toSeq.map(Bar.apply)
  )
   .title("Ejemplo")
   .xAxis(aux.map(_.getInt(0)))
   .yAxis()
   .frame()
   .bottomLegend()
   .render()